In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pickle
import numpy as np
import torch

In [ ]:
datelist = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27] ## Consider only weekdays (Mon–Fri) for the month of April.
datalength = 144 * len(datelist)
print(f'Dataset contains total {len(datelist)} days, and the length is {datalength}.')

In [ ]:
def name_call(category, day, order):  ## category: "in_flow" or "out_flow"
    directory = './Mapmatching_output/18-04-'+format(day, '02')+'/'
    filename = str(category)+'_'+format(day, '02')+'_'+format(order, '03')+'.pkl'
    full_name = directory + filename
    return full_name

In [ ]:
def get_time_list(num):
    time_list = [num + 144*i for i in range(-40, 40)]
    valid_list = [value for value in time_list if value >= 0 and value < datalength and value!=num]
    return valid_list

## Load inflow data and Generate inflow adj matrix

In [ ]:
edge_number = 451 # You should input the exact nodes number of the study area network 
adj = np.zeros(shape=(datalength, edge_number, edge_number))
count = 0

for i in datelist:
    for j in range(0, 144):
        try:
            dataname = name_call('in_flow', i, j)
            data = np.array(torch.load(dataname), dtype=np.float32)
            adj[count] = data
            count += 1
        except FileNotFoundError:
            count += 1
            continue
if count == datalength:
    print(f'Total {count} data have been successfully loaded.')
else:
    print(f'The loading process has something wrong.\nThe number of data should be {datalength}.')

In [ ]:
%%time

for i in range(0, datalength):
    if np.max(adj[i]) == 0:
        try:
            if np.max(adj[i+1]) != 0:
                adj[i] = np.round((adj[i-1] + adj[i+1])/2)
            else:
                raise Exception("Exception process should be conducted.")
        except:
            time_list = get_time_list(i)
            length = 0
            for k in time_list:
                if np.max(adj[k]) != 0:
                    length += 1
                    adj[i] += adj[k]
            adj[i] = np.round(adj[i]/length)

In [ ]:
counts = 0
missing_list = []
for i in range(0, datalength):
    if np.max(adj[i]) == 0:
        counts += 1
        missing_list.append(i)
print(f'The number of missing data is {counts}.')
print(missing_list)
## The counts value should be 0

In [ ]:
adj.shape

In [ ]:
inflow_adj = np.copy(adj)
np.save('../../input_data/inflow_adj_data.npy',inflow_adj)

## Load outflow data and generate outflow adj matrix

In [ ]:
adj = np.zeros(shape=(datalength, edge_number, edge_number))
count = 0

for i in datelist:
    for j in range(0, 144):
        try:
            dataname = name_call('out_flow', i, j)
            data = np.array(torch.load(dataname), dtype=np.float32)
            adj[count] = data
            count += 1
        except FileNotFoundError:
            count += 1
            continue
if count == datalength:
    print(f'Total {count} data have been successfully loaded.')
else:
    print(f'The loading process has something wrong.\nThe number of data should be {datalength}.')

In [ ]:
%%time

for i in range(0, datalength):
    if np.max(adj[i]) == 0:
        try:
            if np.max(adj[i+1]) != 0:
                adj[i] = np.round((adj[i-1] + adj[i+1])/2)
            else:
                raise Exception("Exception process should be conducted.")
        except:
            time_list = get_time_list(i)
            length = 0
            for k in time_list:
                if np.max(adj[k]) != 0:
                    length += 1
                    adj[i] += adj[k]
            adj[i] = np.round(adj[i]/length)

In [ ]:
counts = 0
missing_list = []
for i in range(0, datalength):
    if np.max(adj[i]) == 0:
        counts += 1
        missing_list.append(i)
print(f'The number of missing data is {counts}.')
print(missing_list)
## The counts value should be 0

In [ ]:
%%time

outflow_adj = np.copy(adj)
np.save('../../input_data/outflow_adj_data.npy',outflow_adj)